In [ ]:
# 將Json跟XML的標註檔轉成grounding dino 訓練用的格式
import json
import jsonlines
from glob import glob
import xml.etree.ElementTree as ET
import os
from PIL import Image
import cv2
import shutil


j_paths = glob('D:\\yangu\\dataset\\base_GD_data\\muti\\*.json')
x_paths = glob('D:\\yangu\\dataset\\base_GD_data\\muti\\*.xml')
metas = []

for j_file in j_paths:
    data = json.load(open(j_file,'r'))
    filename = data["imagePath"]
    if os.path.isfile('D:\\yangu\\dataset\\base_GD_data\\Images\\'+filename):
        image = cv2.imread('D:\\yangu\\dataset\\base_GD_data\\Images\\'+filename)
        # print('D:\\yangu\\dataset\\base_GD_data\\Images\\'+filename)
        shutil.copy('D:\\yangu\\dataset\\base_GD_data\\Images\\'+filename,'D:\\yangu\\dataset\\base_GD_data\\img\\'+filename )
        
        if image is not None:
            height = data["imageHeight"]
            width = data["imageWidth"]
            instance_list = []
            # ddict = {"version": "4.2.9", "flags": {}, "shapes": [{"label": "840", "text": "", "points": [[295.88851351351354, 269.97297297297297], [266.8344594594595, 304.4324324324324], [270.3817567567568, 307.81081081081084], [299.60472972972974, 271.66216216216213], [299.60472972972974, 271.66216216216213]], "group_id": null, "shape_type": "polygon", "flags": {}}], "imagePath": "100_1_F6362192AM0500011.jpg", "imageData": null, "imageHeight": 480, "imageWidth": 640}
            for d in data["shapes"]:
                seg = list(zip(*d["points"]))
                bbox_xyxy = [round(min(seg[0]),2), round(min(seg[1]),2), round(max(seg[0]), 2), round(max(seg[1]),2)]
                instance_list.append({
                    "bbox": bbox_xyxy,
                    "label": 0,
                    "category": 'defect'})
            metas.append(
            {
                "filename": filename,
                "height": height,
                "width": width,
                "detection": {
                    "instances": instance_list
                }
            })

for x_path in x_paths:
    try:
        instance_list = []
        tree = ET.parse(x_path)
        root = tree.getroot()
        bndboxes = root.findall(".//bndbox")
        filename = x_path.split('\\')[-1].replace('xml','jpg')
        if os.path.isfile('D:\\yangu\\dataset\\base_GD_data\\Images\\'+filename):
            shutil.copy('D:\\yangu\\dataset\\base_GD_data\\Images\\'+filename,'D:\\yangu\\dataset\\base_GD_data\\img\\'+filename )
            image = cv2.imread('D:\\yangu\\dataset\\base_GD_data\\Images\\'+filename)
            if image[0:20, 0:20] is not None:
                height = root.find("size/height").text
                width = root.find("size/width").text
                for bndbox in bndboxes:
                    xmin = int(bndbox.find("xmin").text)
                    ymin = int(bndbox.find("ymin").text)
                    xmax = int(bndbox.find("xmax").text)
                    ymax = int(bndbox.find("ymax").text)
                    bbox_xyxy = [xmin, ymin, xmax, ymax]
                    instance_list.append({
                        "bbox": bbox_xyxy,
                        "label": 0,
                        "category": 'defect'})
                metas.append(
                {
                    "filename": filename,
                    "height": height,
                    "width": width,
                    "detection": {
                        "instances": instance_list
                    }
                })
    except:
        pass

with jsonlines.open("D:\\GroundingDINO\\Open-GroundingDino\\data\\od_data.json", mode="w") as writer:
    writer.write_all(metas)